In [1]:
import os
import sys
sys.path.insert(0, "../")

In [2]:
from core.nl_parser import SYSTEM, USER, NLParser

# Setup

In [3]:
open_ai_key = os.environ["OPENAI_API_KEY"]
antropic_key = os.environ["ANTHROPIC_API_KEY"]

In [4]:
# nl_parser = NLParser(open_ai_key)

In [5]:
# !pip install \
#   "pydantic>=2.7,<3" \
#   "langchain>=0.3,<0.4" \
#   "langchain-core>=0.3,<0.4" \
#   "langchain-community>=0.3,<0.4" \
#   "langchain-openai>=0.2,<0.3" \
#   "langchain-anthropic>=0.2,<0.3" \
#   "openai>=1.40" \
#   "anthropic>=0.30"

# Generate

In [6]:
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [23]:
ENDPOINT="us-central1-aiplatform.googleapis.com"
REGION="us-central1"
PROJECT_ID="support-robot-448808"
BUCKET_NAME="base_bucket-2025"
BUCKET_URI = f"gs://{BUCKET_NAME}"
MODEL_LOCATION = "us-central1"

In [31]:
from langchain_google_vertexai import ChatVertexAI
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION) 

In [7]:
duration = 90
focus = "Endurance"
prefs = ""

In [17]:
model = "gpt-5-mini"
llm_open_ai = ChatOpenAI(model=model, temperature=1, api_key=open_ai_key)

In [18]:
model = "claude-sonnet-4-5-20250929"
llm_claude = ChatAnthropic(model=model, temperature=1, api_key=antropic_key)

In [48]:
model = "gemini-2.5-flash-lite"
llm_vertex = ChatVertexAI(model=model, temperature=1.0)

In [45]:
prompt = ChatPromptTemplate.from_messages([("system", SYSTEM), ("user", USER)])

In [20]:
%%time
msg = prompt.format_messages(duration_min=duration, focus=focus, prefs=prefs or "—")
raw = llm_open_ai.invoke(msg)
print(raw)

content='{\n  "name": "90 min Steady Endurance with Short Recoveries",\n  "focus": "Endurance",\n  "description": "A 90-minute aerobic endurance session built around long, steady Z2 blocks with brief low-intensity recoveries to maintain consistency and focus. Suited for ambitious cyclists to strengthen aerobic capacity and muscular endurance without surges or high-intensity stress.",\n  "steps": [\n    {\n      "duration_s": 600,\n      "pct_ftp": 0.50,\n      "pct_ftp_end": 0.70,\n      "kind": "warmup",\n      "note": "Progressive warm-up: raise blood flow and cadence from easy Z1 into low Z3/Z2 to prepare for sustained work."\n    },\n    {\n      "duration_s": 1200,\n      "pct_ftp": 0.66,\n      "kind": "steady",\n      "note": "Main Z2 block: settle into steady aerobic pace to build sustained power and efficient pedal mechanics."\n    },\n    {\n      "duration_s": 120,\n      "pct_ftp": 0.55,\n      "kind": "steady",\n      "note": "Short recovery: spin easy in low Z2/Z1 to reco

In [35]:
%%time
msg = prompt.format_messages(duration_min=duration, focus=focus, prefs=prefs or "—")
raw = llm_claude.invoke(msg)
print(raw)

content='```json\n{\n  "name": "Steady State Endurance",\n  "focus": "Endurance",\n  "description": "A classic aerobic endurance session designed to build mitochondrial density and fat oxidation capacity. The sustained Zone 2 effort develops the aerobic base crucial for long rides and recovery between high-intensity blocks. Perfect for building durability without creating excessive fatigue.",\n  "steps": [\n    {"duration_s": 300, "pct_ftp": 0.50, "pct_ftp_end": 0.60, "kind": "warmup", "note": "Easy spin to wake up the legs and prepare joints and muscles"},\n    {"duration_s": 300, "pct_ftp": 0.60, "pct_ftp_end": 0.68, "kind": "warmup", "note": "Gradually increase effort into Zone 2, focusing on smooth pedaling"},\n    {"duration_s": 240, "pct_ftp": 0.68, "pct_ftp_end": 0.72, "kind": "warmup", "note": "Final warmup ramp to settle into comfortable Zone 2 rhythm"},\n    {"duration_s": 1200, "pct_ftp": 0.72, "kind": "steady", "note": "First main block—find your sustainable aerobic pace, c

In [49]:
%%time
msg = prompt.format_messages(duration_min=duration, focus=focus, prefs=prefs or "—")
raw = llm_vertex.invoke(msg)
print(raw)

content='```json\n{\n "name": "Progressive Endurance Build",\n "focus": "Endurance",\n "description": "This workout focuses on building aerobic base by progressively increasing the duration and intensity within the endurance zone. It aims to improve sustained power output and fatigue resistance, crucial for ambitious cyclists targeting long events or general fitness.",\n "steps": [\n  {"duration_s": 600, "pct_ftp": 0.50, "kind": "warmup", "pct_ftp_end": 0.55, "note": "Start easy spinning to gradually raise heart rate and warm up the legs."},\n  {"duration_s": 600, "pct_ftp": 0.55, "kind": "warmup", "pct_ftp_end": 0.60, "note": "Increase cadence and power slightly to prepare for endurance efforts."},\n  {"duration_s": 1200, "pct_ftp": 0.65, "kind": "steady", "note": "First sustained endurance block. Focus on maintaining a smooth pedal stroke and steady power output."},\n  {"duration_s": 600, "pct_ftp": 0.60, "kind": "steady", "note": "Recovery period. Spin easy to allow legs to recover 